# Preparation

In [3]:
import os
print('current direc:',os.getcwd())

current direc: /content


In [2]:
!pip install transformers
!pip3 -q install emoji

     |████████████████████████████████| 4.0 MB 44.1 MB/s 
     |████████████████████████████████| 6.6 MB 59.4 MB/s 
     |████████████████████████████████| 77 kB 8.7 MB/s 
     |████████████████████████████████| 895 kB 76.4 MB/s 
     |████████████████████████████████| 596 kB 71.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 175 kB 38.0 MB/s 


# Bertweet for textfeature

In [4]:
%cd ./drive/MyDrive/Colab\ Notebooks/pytorch-multimodal_sarcasm_detection

/content/drive/MyDrive/Colab Notebooks/pytorch-multimodal_sarcasm_detection


In [5]:
import torch 
from transformers import AutoModel, AutoTokenizer
from transformers import (get_linear_schedule_with_warmup,AdamW,AutoModel, AutoTokenizer, AutoModelForSequenceClassification)
from torch.utils.data import (TensorDataset,DataLoader, RandomSampler, SequentialSampler, Dataset)
import ast
import pandas as pd

from sklearn.utils import shuffle

import os
import random
import time
import datetime
import torch
import argparse
import numpy as np
import pandas as pd
from torch.nn import functional as F

In [6]:
model_1 = torch.load('./berttweet_2epoch.pt')

In [7]:
def load_data_lists(path):
    data_points_lists = []
    with open(path, encoding='utf-8') as f:
        lines = f.readlines()

        for line in lines:
            try:
                data_points_lists.append(ast.literal_eval(line))
            except:
                # Ignore lines with errors
                pass

    print('Found {} lines in "{}".'.format(len(lines), path))
    print('Successfully loaded {} data points from "{}".'.format(len(data_points_lists), path))
    
    return data_points_lists

COLUMN_NAMES = ['ID', 'Text', 'Sarcastic']

In [8]:
def construct_df(data_points_lists, column_names=COLUMN_NAMES):
    df = pd.DataFrame(data_points_lists, columns=column_names)
    df['ID'] = pd.to_numeric(df['ID'])
    df['Sarcastic'] = df['Sarcastic'].astype('bool')
    
    return df

In [9]:
train_df = construct_df(load_data_lists('text_data/train.txt'))

train_df.info()
train_df.head()

Found 29040 lines in "text_data/train.txt".
Successfully loaded 29040 data points from "text_data/train.txt".
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29040 entries, 0 to 29039
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         29040 non-null  int64 
 1   Text       29040 non-null  object
 2   Sarcastic  29040 non-null  bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 482.2+ KB


,ID,Text,Sarcastic
0,910308516510011393,most # funny quotes : 21 snarky and # funny qu...,True
1,725333760762363905,spurs # creativethinking ! <url>,True
2,840006160660983809,<user> thanks for showing up for our appointme...,True
3,854334602516733952,only a hardcore fan of sir jonny sins will get...,True
4,908913372199915520,haha . # lol,True


In [10]:
test_df = construct_df(load_data_lists('text_data/test.txt'), column_names=COLUMN_NAMES + ['Sarc_2'])

test_df.info()
test_df.head()

Found 2409 lines in "text_data/test.txt".
Successfully loaded 2409 data points from "text_data/test.txt".
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409 entries, 0 to 2408
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         2409 non-null   int64 
 1   Text       2409 non-null   object
 2   Sarcastic  2409 non-null   bool  
 3   Sarc_2     2409 non-null   int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 58.9+ KB


,ID,Text,Sarcastic,Sarc_2
0,862902619928506372,i am guessing # netflix no longer lets you gra...,True,1
1,892551658487631873,it 's the insensitive strikeouts at suntrust p...,True,1
2,853143461360480256,"following the path of the river calder , so .....",True,1
3,918423568823840768,# westernsahara # authority has no lessons 2ge...,True,1
4,731617467718610944,hey <user> great sale !,True,1


In [11]:
valid_df = construct_df(load_data_lists('text_data/valid.txt'), column_names=COLUMN_NAMES + ['Sarc_2'])

valid_df.info()
valid_df.head()

Found 2410 lines in "text_data/valid.txt".
Successfully loaded 2410 data points from "text_data/valid.txt".
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         2410 non-null   int64 
 1   Text       2410 non-null   object
 2   Sarcastic  2410 non-null   bool  
 3   Sarc_2     2410 non-null   int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 59.0+ KB


,ID,Text,Sarcastic,Sarc_2
0,915657464401580032,whew ... that extra <num> miles today to the g...,True,1
1,854678856724340736,""" oh , good . now no one will know we 're here...",True,1
2,904892917277274112,how much of it you think is true ? has this be...,True,1
3,855466461296504832,<user> finally found proof that the earth is f...,True,1
4,927373534652805120,many ways to overcome tension & fear but nothi...,True,1


In [12]:
train_df = shuffle(train_df, random_state=42)
valid_df = shuffle(valid_df, random_state=42)
test_df = shuffle(test_df, random_state=42)

In [ ]:
def bert_encode(df, tokenizer):
    input_ids = []
    attention_masks = []
    # print(df)
    for sent in df[['Text']].values:
        sent = sent.item()
        encoded_dict = tokenizer.encode_plus(
                            sent,                      
                            add_special_tokens = True, 
                            max_length = 128,           
                            pad_to_max_length = True,
                            truncation = True,
                            return_attention_mask = True,   
                            return_tensors = 'pt',    
                    )
           
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    inputs = {
    'input_word_ids': input_ids,
    'input_mask': attention_masks}

    return inputs

In [ ]:
def prepare_dataloaders(train_df,test_df,batch_size=64):
    # Load the AutoTokenizer with a normalization mode if the input Tweet is raw
    
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
    
    tweet_train = bert_encode(train_df, tokenizer)
    tweet_train_labels = train_df['Sarcastic'].astype(int)
    
    tweet_test = bert_encode(test_df, tokenizer)

    input_ids, attention_masks = tweet_train.values()
    labels = torch.tensor(tweet_train_labels.values)
    train_dataset = TensorDataset(input_ids, attention_masks, labels)

    
    input_ids, attention_masks = tweet_test.values()
    test_dataset = TensorDataset(input_ids, attention_masks)

    
    train_dataloader = DataLoader(
                train_dataset,
                sampler = RandomSampler(train_dataset), 
                batch_size = batch_size 
            )


    test_dataloader = DataLoader(
                test_dataset, 
                sampler = SequentialSampler(test_dataset), 
                batch_size = batch_size
            )
    return train_dataloader, test_dataloader

In [ ]:
def predict(model,test_dataloader):
    model.eval()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    preds = []

    for batch in test_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask)
            logits = outputs.logits

        logits = logits.detach().cpu().numpy()
        for logit in logits:
            preds.append(logit)
    # preds
    return outputs

In [ ]:
train_dataloader,test_dataloader = prepare_dataloaders(train_df, test_df)
_,val_dataloader = prepare_dataloaders(train_df, valid_df)

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
result_1 = predict(model_1,test_dataloader)

In [ ]:
result_1

SequenceClassifierOutput([('logits', tensor([[ 0.7634, -0.7053],
                                   [ 2.5817, -2.6180],
                                   [ 2.3406, -2.3473],
                                   [ 2.1437, -2.1356],
                                   [ 2.7959, -2.7942],
                                   [ 2.9086, -2.9929],
                                   [-2.5752,  2.6426],
                                   [ 2.6114, -2.6237],
                                   [ 2.5706, -2.6024],
                                   [ 2.0115, -1.9628],
                                   [-1.7635,  1.8254],
                                   [-0.8935,  1.0105],
                                   [ 2.6490, -2.6544],
                                   [-3.2300,  3.2771],
                                   [ 2.3816, -2.3569],
                                   [ 0.8064, -0.7265],
                                   [-1.6984,  1.7258],
                                   [ 1.5627, -1.5105],


In [ ]:
test_df['Sarcastic']

2009    False
2082    False
1684    False
2301    False
792      True
        ...  
1638    False
1095    False
1130    False
1294    False
860      True
Name: Sarcastic, Length: 2409, dtype: bool

# Fusion model for image and attribute

In [5]:
%cd ./drive/MyDrive/Colab\ Notebooks/pytorch-multimodal_sarcasm_detection

/content/drive/MyDrive/Colab Notebooks/pytorch-multimodal_sarcasm_detection


# download

In [6]:
!gdown --id 1dJERrVlp7DlNSXk-uvbbG6Rv7uvqTOKd

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1dJERrVlp7DlNSXk-uvbbG6Rv7uvqTOKd
To: /content/drive/MyDrive/Colab Notebooks/pytorch-multimodal_sarcasm_detection/dataset_image.z01
100% 734M/734M [00:04<00:00, 161MB/s]


In [7]:
!gdown --id 1pODuKC4gP6-QDQonG8XTqI8w8ds68mE3

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1pODuKC4gP6-QDQonG8XTqI8w8ds68mE3
To: /content/drive/MyDrive/Colab Notebooks/pytorch-multimodal_sarcasm_detection/dataset_image.z02
100% 734M/734M [00:04<00:00, 181MB/s]


In [8]:
!gdown --id 1mK0Nf-jv_h2bgHUCRM4_EsdTiiitZ_Uj

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1mK0Nf-jv_h2bgHUCRM4_EsdTiiitZ_Uj
To: /content/drive/MyDrive/Colab Notebooks/pytorch-multimodal_sarcasm_detection/dataset_image.z03
100% 734M/734M [00:14<00:00, 51.6MB/s]


In [9]:
!gdown --id 1AOWzlOz5hmdO39dEmzhQ4z_nabgzi7Tu

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1AOWzlOz5hmdO39dEmzhQ4z_nabgzi7Tu
To: /content/drive/MyDrive/Colab Notebooks/pytorch-multimodal_sarcasm_detection/dataset_image.zip
100% 460M/460M [00:03<00:00, 144MB/s]


In [10]:
!zip -FF dataset_image.zip --out dataset_image_full.zip

Streaming output truncated to the last 5000 lines.
 copying: dataset_image/851584901593931776.jpg  (94469 bytes)
 copying: dataset_image/851589786385223680.jpg  (120741 bytes)
 copying: dataset_image/851623486648754176.jpg  (86739 bytes)
 copying: dataset_image/851638105492910080.jpg  (54081 bytes)
 copying: dataset_image/851668637681422337.jpg  (31223 bytes)
 copying: dataset_image/851683467209441282.jpg  (137690 bytes)
 copying: dataset_image/851688823708889088.jpg  (97590 bytes)
 copying: dataset_image/851704924568268800.jpg  (54555 bytes)
 copying: dataset_image/851747075129606145.jpg  (29953 bytes)
 copying: dataset_image/851754779046944768.jpg  (66626 bytes)
 copying: dataset_image/851756040895049728.jpg  (191551 bytes)
 copying: dataset_image/851774723772080128.jpg  (92185 bytes)
 copying: dataset_image/851781920442781698.jpg  (37671 bytes)
 copying: dataset_image/851797551254601728.jpg  (22441 bytes)
 copying: dataset_image/851829550732464129.jpg  (12839 bytes)
 copying: datase

In [11]:
!unzip dataset_image_full.zip 

Streaming output truncated to the last 5000 lines.
  inflating: dataset_image/851580029368647680.jpg  
  inflating: dataset_image/851581265782243328.jpg  
  inflating: dataset_image/851584901593931776.jpg  
  inflating: dataset_image/851589786385223680.jpg  
  inflating: dataset_image/851623486648754176.jpg  
  inflating: dataset_image/851638105492910080.jpg  
  inflating: dataset_image/851668637681422337.jpg  
  inflating: dataset_image/851683467209441282.jpg  
  inflating: dataset_image/851688823708889088.jpg  
  inflating: dataset_image/851704924568268800.jpg  
  inflating: dataset_image/851747075129606145.jpg  
  inflating: dataset_image/851754779046944768.jpg  
  inflating: dataset_image/851756040895049728.jpg  
  inflating: dataset_image/851774723772080128.jpg  
  inflating: dataset_image/851781920442781698.jpg  
  inflating: dataset_image/851797551254601728.jpg  
  inflating: dataset_image/851829550732464129.jpg  
  inflating: dataset_image/851853031977218048.jpg  
  inflating: 

In [12]:
!gdown --id 1YZgCVbGgqNIpwU1XLblV5APNJIdBFHED

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1YZgCVbGgqNIpwU1XLblV5APNJIdBFHED
To: /content/drive/MyDrive/Colab Notebooks/pytorch-multimodal_sarcasm_detection/image_feature_data.z01
  4% 157M/4.29G [00:04<01:06, 62.3MB/s]Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/gdown/cli.py", line 166, in main
    resume=args.continue_,
  File "/usr/local/lib/python3.7/dist-packages/gdown/download.py", line 258, in download
    f.write(chunk)
KeyboardInterrupt
  4% 166M/4.29G [00:04<01:59, 34.5MB/s]


In [ ]:
!gdown --id 1l9WAi6NHWB5ip6l1cKaHkYB-vybsSWpO

Permission denied: https://drive.google.com/uc?id=1l9WAi6NHWB5ip6l1cKaHkYB-vybsSWpO
Maybe you need to change permission over 'Anyone with the link'?


In [ ]:
!gdown --id 1t84oEZQxsXwCiDRcgAfN8t76k6lwCcDS

Permission denied: https://drive.google.com/uc?id=1t84oEZQxsXwCiDRcgAfN8t76k6lwCcDS
Maybe you need to change permission over 'Anyone with the link'?


In [ ]:
!gdown --id 1Tz5e1z_o36tf32vQrvcib8ga6oUuVtKw

Permission denied: https://drive.google.com/uc?id=1Tz5e1z_o36tf32vQrvcib8ga6oUuVtKw
Maybe you need to change permission over 'Anyone with the link'?


In [ ]:
!gdown --id 1FTRjEY0f4V0nZbNhBGJXzSfUBi1CPNhx

Permission denied: https://drive.google.com/uc?id=1FTRjEY0f4V0nZbNhBGJXzSfUBi1CPNhx
Maybe you need to change permission over 'Anyone with the link'?


In [ ]:
!gdown --id 1Tj9oOiS-tTZQ3De-eIGSKVlib0OZDEbP

Permission denied: https://drive.google.com/uc?id=1Tj9oOiS-tTZQ3De-eIGSKVlib0OZDEbP
Maybe you need to change permission over 'Anyone with the link'?


In [ ]:
!gdown --id 1QpCfNpWLCX_V9HHE15mdLNbdhai1Rimd

Permission denied: https://drive.google.com/uc?id=1QpCfNpWLCX_V9HHE15mdLNbdhai1Rimd
Maybe you need to change permission over 'Anyone with the link'?


In [ ]:
!gdown --id 1URz72Q_sibnT0Fdyu6tFH6sh8BYVniro

Permission denied: https://drive.google.com/uc?id=1URz72Q_sibnT0Fdyu6tFH6sh8BYVniro
Maybe you need to change permission over 'Anyone with the link'?


# Fusion model

In [13]:
import shutil

In [14]:
os.mkdir('dataset_image/test')
for id in test_df['ID']:
  img_path = 'dataset_image/{}.jpg'.format(id)
  if os.path.isfile(img_path):
    shutil.move(img_path, 'dataset_image/test/{}.jpg'.format(id))

In [15]:
os.mkdir('dataset_image/train')
for id in train_df['ID']:
  img_path = 'dataset_image/{}.jpg'.format(id)
  if os.path.isfile(img_path):
    shutil.move(img_path, 'dataset_image/train/{}.jpg'.format(id))

In [16]:
os.mkdir('dataset_image/valid')
for id in valid_df['ID']:
  img_path = 'dataset_image/{}.jpg'.format(id)
  if os.path.isfile(img_path):
    shutil.move(img_path, 'dataset_image/valid/{}.jpg'.format(id))

In [17]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader,random_split
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import PIL
import pickle

In [18]:
WORKING_PATH=os.getcwd()
TEXT_LENGTH=75
TEXT_HIDDEN=256

In [21]:
!python LoadData.py

Traceback (most recent call last):
  File "LoadData.py", line 156, in <module>
    train_loader = DataLoader(train_set,batch_size=batch_size, shuffle=True)
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 277, in __init__
    sampler = RandomSampler(dataset, generator=generator)  # type: ignore[arg-type]
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/sampler.py", line 98, in __init__
    "value, but got num_samples={}".format(self.num_samples))
ValueError: num_samples should be a positive integer value, but got num_samples=0


In [ ]:
!python AttributeFeature.py

torch.Size([32, 200])
torch.Size([32, 5, 200])
